In [18]:
import gensim
import pandas as pd
from nltk.stem import PorterStemmer
from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
df = pd.read_json("Sports_and_Outdoors_5.json", lines=True)
df.head(5)
df.reviewText.loc[3]

'This works no better than any 3/32 punch you would find at the hardware store. Actually, I think you would be better with a regular punch as it has more to hold on to.'

In [3]:
df.shape

(296337, 9)

In [21]:
check_df = df.head(200000)
check_df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AIXZKN4ACSKI,1881509818,David Briner,"[0, 0]",This came in on time and I am veru happy with ...,5,Woks very good,1390694400,"01 26, 2014"
1,A1L5P841VIO02V,1881509818,Jason A. Kramer,"[1, 1]",I had a factory Glock tool that I was using fo...,5,Works as well as the factory tool,1328140800,"02 2, 2012"
2,AB2W04NI4OEAD,1881509818,J. Fernald,"[2, 2]",If you don't have a 3/32 punch or would like t...,4,"It's a punch, that's all.",1330387200,"02 28, 2012"
3,A148SVSWKTJKU6,1881509818,"Jusitn A. Watts ""Maverick9614""","[0, 0]",This works no better than any 3/32 punch you w...,4,It's a punch with a Glock logo.,1328400000,"02 5, 2012"
4,AAAWJ6LW9WMOO,1881509818,Material Man,"[0, 0]",I purchased this thinking maybe I need a speci...,4,"Ok,tool does what a regular punch does.",1366675200,"04 23, 2013"
...,...,...,...,...,...,...,...,...,...
199995,A15B3F1CO7JBYW,B003Y5AGCA,Tim Schrage,"[0, 0]",Solidly enough made to accomodate a Hi-Point ....,4,Works great,1381363200,"10 10, 2013"
199996,A3KCX0DU4ZF439,B003Y5C132,Apex Predator,"[0, 4]",Nothing to really say. It looks like the pictu...,5,Perfect,1318464000,"10 13, 2011"
199997,A3V16CPB7YE3VD,B003Y5C132,A-WAX,"[2, 2]",Arrived on time. The inside looks wonderful. W...,1,RUSTED OUT!,1386028800,"12 3, 2013"
199998,A379VGIWYYYK2M,B003Y5C132,BackBone,"[0, 0]",Holds all my ammo just perfect. I put 12 gage ...,5,Perfect.,1371945600,"06 23, 2013"


In [22]:
review_text = check_df.reviewText.progress_apply(gensim.utils.simple_preprocess)
review_text

  0%|          | 0/200000 [00:00<?, ?it/s]

0         [this, came, in, on, time, and, am, veru, happ...
1         [had, factory, glock, tool, that, was, using, ...
2         [if, you, don, have, punch, or, would, like, t...
3         [this, works, no, better, than, any, punch, yo...
4         [purchased, this, thinking, maybe, need, speci...
                                ...                        
199995    [solidly, enough, made, to, accomodate, hi, po...
199996    [nothing, to, really, say, it, looks, like, th...
199997    [arrived, on, time, the, inside, looks, wonder...
199998    [holds, all, my, ammo, just, perfect, put, gag...
199999    [watertight, has, nice, solid, rubber, seal, w...
Name: reviewText, Length: 200000, dtype: object

In [23]:
stemmer = PorterStemmer()
stemmed = review_text.progress_apply(lambda x: [stemmer.stem(y) for y in x])
stemmed

  0%|          | 0/200000 [00:00<?, ?it/s]

0         [thi, came, in, on, time, and, am, veru, happi...
1         [had, factori, glock, tool, that, wa, use, for...
2         [if, you, don, have, punch, or, would, like, t...
3         [thi, work, no, better, than, ani, punch, you,...
4         [purchas, thi, think, mayb, need, special, too...
                                ...                        
199995    [solidli, enough, made, to, accomod, hi, point...
199996    [noth, to, realli, say, it, look, like, the, p...
199997    [arriv, on, time, the, insid, look, wonder, wi...
199998    [hold, all, my, ammo, just, perfect, put, gage...
199999    [watertight, ha, nice, solid, rubber, seal, wo...
Name: reviewText, Length: 200000, dtype: object

In [24]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=3,
)

In [32]:
model.build_vocab(stemmed, progress_per=1000)

In [34]:
model.corpus_count

200000

In [35]:
model.train(stemmed, total_examples=model.corpus_count, epochs=model.epochs)

(59025043, 79348500)

In [36]:
model.save("./word2vec_test.model")

In [46]:
model.wv.most_similar("good")

[('decent', 0.863539457321167),
 ('great', 0.7678033113479614),
 ('nice', 0.6867431998252869),
 ('fantast', 0.6789023876190186),
 ('excel', 0.6010145545005798),
 ('high', 0.5832275152206421),
 ('awesom', 0.5528854727745056),
 ('superb', 0.5227035284042358),
 ('sweet', 0.5150555372238159),
 ('fair', 0.512129008769989)]

In [41]:
model.wv.similarity(w1="good", w2="great")

0.7678033

In [45]:
model.wv.similarity(w1="slow", w2="eight")

-0.07017534